These are the preprocessing steps I'm doing for this dataset
1. to check whether dataset contains nulls, in this there are no null values
2. dataset is balanced or not, I'm having balanced dataset
3. Dropping unwanted columns, In this dataset 'ids','date','flag','user' are not depending on output column so I'm dropping those columns
4. To check whether we are having duplicates If so we are dropping duplicate rows in this dataset
5. preprocessing the text column

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import contractions
from num2words import num2words
from textblob import TextBlob
from spellchecker import SpellChecker
import re
from decimal import Decimal
import plotly.express as px
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


column_names = ['target','ids','date','flag','user','text']

df = pd.read_csv('/users/amulya/Downloads/sentiment_text_analysis.csv',encoding='ISO-8859-1', header=None, names=column_names)
df.head(2)
nulls = df.isnull().sum()
# print(nulls)
print(df['target'].value_counts())
print(df['flag'].value_counts())

# df['target'] = df['target'].replace({4:1})
df.head()

[nltk_data] Downloading package punkt to /Users/amulya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amulya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/amulya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


target
0    800000
4    800000
Name: count, dtype: int64
flag
NO_QUERY    1600000
Name: count, dtype: int64


,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [2]:
unwanted_columns = ['ids','date','flag','user']
df = df.drop(unwanted_columns,axis = 1)

In [3]:
print(df['target'].value_counts())

target
0    800000
4    800000
Name: count, dtype: int64


In [4]:
size, dups = df.size, df.duplicated().sum()
# pie = px.pie(values=[size, dups], names=["Total Size", "Duplicates"], hole=0.4, title="Duplicates vs Total Size")
# pie.show()

In [5]:
df = df.drop_duplicates()

# Get size and count of duplicates
size = df.size
dups = df.duplicated().sum()

# Pie plot
# pie = px.pie(values=[size, dups], names=["Total Size", "Duplicates"], hole=0.4, title="Duplicates vs Total Size")
# pie.show()

In [6]:
print(df['target'].value_counts())

target
4    793506
0    790185
Name: count, dtype: int64


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1583691 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1583691 non-null  int64 
 1   text    1583691 non-null  object
dtypes: int64(1), object(1)
memory usage: 36.2+ MB


In [8]:
df['text'] = df['text'].str.lower() # to convert all the text to lower case

In [9]:
# df['text'] = df['text'].apply(lambda x:re.sub('[\d]','',x))

In [10]:
df["text"] = df["text"].apply(lambda s: ' '.join(re.sub("(w+://S+)", " ", s).split()))  # to remove links

In [11]:
df['text'] = df['text'].apply(word_tokenize) # Tokenization (splitting a sentence in to tokens)

In [12]:
# punctuation = set(string.punctuation)
# df['text'] = df['text'].apply(lambda tokens: [token for token in tokens if token not in punctuation])

In [13]:
# stop_words = set(stopwords.words('english'))
# df['text'] = df['text'].apply(lambda tokens: [token for token in tokens if token not in stop_words])

In [14]:
lemmatizer = WordNetLemmatizer()
df['text'] = df['text'].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens]) #Applying lemmatization

In [15]:
df['text'] = df['text'].apply(lambda tokens: [contractions.fix(token) for token in tokens]) # (don't -> do not)

In [16]:
df['text'] = df['text'].apply(lambda tokens: [token for token in tokens if token.isalnum()]) # removing special characters

In [17]:
df['text'] = df['text'].apply(lambda tokens: [token.strip() for token in tokens]) # removing extra spaces

In [18]:
# df['text'] = df['text'].apply(lambda tokens: [num2words(token) if isinstance(token, int) else token if isinstance(token, Decimal) else token for token in tokens])

In [19]:
df['text'] = df['text'].apply(lambda tokens: ' '.join(tokens)) # combining all these to single

In [20]:
df['text'].head()

0    switchfoot http awww that a bummer you shoulda...
1    is upset that he ca update his facebook by tex...
2    kenichan i dived many time for the ball manage...
3         my whole body feel itchy and like it on fire
4    nationwideclass no it not behaving at all i ma...
Name: text, dtype: object

Below I'm inporting keras from tensorflow
1. I'm having large size of dataset so I'm taking 200000 sample of data 

In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, LSTM, SpatialDropout1D, Conv1D, Bidirectional, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import one_hot

class_0 = df[df['target'] == 0]
class_1 = df[df['target'] == 4]
print(len(class_0))
print(len(class_1))
sampled_class_0 = class_0.sample(n=100000, random_state=42)
sampled_class_1 = class_1.sample(n=100000, random_state=42)
new_df = pd.concat([sampled_class_0, sampled_class_1])
new_df = new_df.sample(frac=1, random_state=42).reset_index(drop=True)
new_df.info()
text = new_df['text'].tolist()
target = new_df['target'].tolist()

790185
793506
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   target  200000 non-null  int64 
 1   text    200000 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.1+ MB


Below I'm replacing output column classes with 1 for positive 

In [23]:
new_df['target'].value_counts()
new_df['target'] = new_df['target'].replace({4:1})
new_df['target'].value_counts()

target
1    100000
0    100000
Name: count, dtype: int64

Below I'm coverting text to embeddings

In [24]:
import numpy as np
vocab_size = 30 # creates embeddings lessthan 30
encoded_reviews = [one_hot(d, vocab_size) for d in new_df['text']]
# print(encoded_reviews)
padded_reviews = pad_sequences(encoded_reviews, maxlen=20, padding='post') # maxlen - 20 : no. of columns
print(len(padded_reviews))
sequence_lengths = [len(seq) for seq in padded_reviews]
print(f'Maximum sequence length: {max(sequence_lengths)}')
print(f'Average sequence length: {np.mean(sequence_lengths)}')


max_len = int(np.percentile(sequence_lengths, 95))
print(max_len)

200000
Maximum sequence length: 20
Average sequence length: 20.0
20


In [25]:
X_train, X_val, y_train, y_val = train_test_split(padded_reviews, new_df['target'], test_size=0.2, random_state=42) # splitting the data in to train and validation data 

Below I'm building neural network by applying embedding layer and other layers

In [26]:
from tensorflow.keras.layers import SimpleRNN, Dropout
from tensorflow.keras import initializers
from tensorflow.keras.initializers import GlorotUniform, he_uniform

embeded_vector_size = 20 # no. of neurons 
model = Sequential()
initializer = he_uniform()
initializer1 = GlorotUniform()
# model.add(Embedding(vocab_size, embeded_vector_size, input_length=20, name="embedding"))
model.add(Embedding(vocab_size, embeded_vector_size, input_length=20))
# model.add(SimpleRNN(256, return_sequences=True, name="simple_rnn"))  # Simple RNN layer
# model.add(LSTM(512, name="lstm"))
# model.add(Dense(512,kernel_initializer = initializer, activation='relu'))
# model.add(Dense(512,kernel_initializer = initializer, activation='relu'))
# model.add(Dense(512,kernel_initializer = initializer, activation='relu'))
# # model.add(Dense(512,kernel_initializer = initializer1, activation='tanh'))
# model.add(Dense(512,kernel_initializer = initializer, activation='relu'))
# model.add(Dense(512,kernel_initializer = initializer, activation='relu'))
# model.add(Dense(512,kernel_initializer = initializer, activation='relu'))
# model.add(Dense(512,kernel_initializer = initializer, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

model.add(SpatialDropout1D(0.2))
model.add(Conv1D(64, 5, activation='relu'))
model.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


/opt/homebrew/anaconda3/envs/amulya_base/lib/python3.9/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [27]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.build(input_shape=(None, 20))

In [28]:
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 20, 20)         │           600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ (None, 20, 20)         │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 16, 64)         │         6,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 402,329 (1.53 MB)

 Trainable params: 402,329 (1.53 MB)

 Non-trainable params: 0 (0.00 B)

None


In [29]:
X_train

array([[ 6, 20, 19, ..., 24, 14,  1],
       [ 5, 11,  4, ...,  0,  0,  0],
       [10, 26, 28, ...,  0,  0,  0],
       ...,
       [16, 17,  6, ...,  0,  0,  0],
       [ 2, 29, 18, ...,  2, 18, 18],
       [ 4,  7, 11, ...,  0,  0,  0]], dtype=int32)

In [30]:
model.fit(padded_reviews, new_df['target'], epochs=4, validation_data=(X_val, y_val))


Epoch 1/4
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 66s 10ms/step - accuracy: 0.5607 - loss: 0.6818 - val_accuracy: 0.5922 - val_loss: 0.6676
Epoch 2/4
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 61s 10ms/step - accuracy: 0.5872 - loss: 0.6695 - val_accuracy: 0.6007 - val_loss: 0.6596
Epoch 3/4
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 63s 10ms/step - accuracy: 0.5978 - loss: 0.6633 - val_accuracy: 0.6064 - val_loss: 0.6560
Epoch 4/4
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 63s 10ms/step - accuracy: 0.6018 - loss: 0.6600 - val_accuracy: 0.6112 - val_loss: 0.6505
